In [1]:
import pandas as pd
from z_report_data.variables import app_vars
from sqlalchemy import create_engine
import json
import os
from dotenv import load_dotenv

load_dotenv()

connection_string = os.getenv("YANZI_CONNECTION_STRING")
engine = create_engine(connection_string)
store_name = app_vars.STORE_NAME

In [2]:
sql = f"select * from transactions where Store = '{store_name}'"
pd.read_sql(sql, engine).to_csv(f'./assets/{store_name}_transactions.csv', index=False)

In [3]:
sql = f"select * from sync_info where store = '{store_name}' and SyncEntityType = 'Transaction'"
pd.read_sql(sql, engine).to_csv(f'./assets/{store_name}_transactions_sync_info.csv', index=False)

In [4]:
sql = (""
       "select td.* "
       "from transaction_discounts td "
       "inner join transactions t on t.id = td.TransactionId "
       f"where t.Store = '{store_name}' and t.DeleteById is null and td.DeleteById is null"
       "")

pd.read_sql(sql, engine).to_csv(f'./assets/{store_name}_transaction_discounts.csv', index=False)

In [5]:
sql = f"select * from store_configurations where StoreName = '{store_name}'"
data = json.loads(pd.read_sql(sql, engine).to_json(orient='records'))
app_vars.header = data[0]['Header']
app_vars.footer = data[0]['Footer']

In [6]:
engine.dispose(close=True)

In [7]:
from datetime import datetime
from z_report_data.variables import app_vars
import pandas as pd

In [8]:
store_name = app_vars.STORE_NAME
transactions = (
    pd
    .read_csv(
        f'./assets/{store_name}_transactions.csv',
        parse_dates=['CreateDate', 'DeleteDate'],
        date_format='%Y-%m-%d %H:%M:%S',
        dtype={'Store': 'category'},
        usecols=['Id', 'CreateDate', 'HeadCount', 'PrivilegeHeadCount', 'Gross', 'Tags', 'DeleteDate', 'SyncInfoId']
    )
    .dropna(how='all')
)

after_start = transactions['CreateDate'] >= app_vars.START_DATE
before_end = transactions['CreateDate'] < app_vars.END_DATE

transactions = transactions[after_start & before_end]

In [9]:
sync_infos = (
    pd
    .read_csv(
        f'./assets/{store_name}_transactions_sync_info.csv',
        usecols=['Id', 'LocalReference', 'ExternalReference'],
        dtype={'ExternalReference': 'int64', 'LocalReference':'int64'}
    )
)

sync_infos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25042 entries, 0 to 25041
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Id                 25042 non-null  int64
 1   ExternalReference  25042 non-null  int64
 2   LocalReference     25042 non-null  int64
dtypes: int64(3)
memory usage: 587.1 KB


In [10]:
discounts = (
    pd
    .read_csv(
        f'./assets/{store_name}_transaction_discounts.csv',
        usecols=['Amount', 'TransactionId', 'Type', 'DeleteDate']
    )
    .where(lambda x : x['DeleteDate'].isna())
)

discounts

,Type,Amount,TransactionId,DeleteDate
0,SeniorDiscount,88.75000,20834,NaN
1,VatExempt,53.25000,20834,NaN
2,SeniorDiscount,341.07143,20835,NaN
3,VatExempt,204.64286,20835,NaN
4,SeniorDiscount,125.44643,20841,NaN
...,...,...,...,...
23858,VatExempt,85.12500,78838,NaN
23859,SeniorDiscount,90.47619,78841,NaN
23860,VatExempt,54.28571,78841,NaN
23861,SeniorDiscount,92.85714,78843,NaN


In [11]:
merged = (
    transactions
    .merge(
        sync_infos,
        how='inner', left_on='SyncInfoId', right_on='Id',
    )
    .dropna(how='all')
    .drop(['Id_y', 'SyncInfoId', 'LocalReference'], axis=1)
    .rename(columns={'Id_x': 'Id'})
)

merged.head()

,Id,HeadCount,PrivilegeHeadCount,Gross,Tags,CreateDate,DeleteDate,ExternalReference
0,20802,2,0,705.0,#DineIn,2023-08-06 09:26:22,NaT,1
1,20803,3,0,485.0,#DineIn,2023-08-06 09:38:24,NaT,2
2,20804,1,0,310.0,#DineIn,2023-08-06 09:49:54,NaT,3
3,20805,1,0,1325.0,#DineIn,2023-08-06 09:58:24,NaT,4
4,20806,3,0,2030.0,#DineIn,2023-08-06 10:32:04,NaT,5


In [12]:
trs = merged
valid = trs[trs['DeleteDate'].isna()]
cancelled = trs[trs['DeleteDate'].isna() == False]

tr_discs = discounts.merge(trs, how='inner', left_on='TransactionId', right_on='Id')
seniors = tr_discs['Type'] == 'SeniorDiscount'
pwds = tr_discs['Type'] == 'Pwd'
priv_discs = tr_discs[seniors | pwds]
opens = tr_discs['Type'] == 'OpenDiscount'
open_discounts = tr_discs[opens]
vat_exes = tr_discs['Type'] == 'VatExempt'
vat_exclusives = tr_discs[vat_exes]

gross = valid['Gross'].sum()
count = valid['Id'].count()
cancelCount = cancelled['Id'].count()
cancelGross = cancelled['Gross'].sum()
priv_disc = priv_discs['Amount'].sum()
open_disc = open_discounts['Amount'].sum()
vat_of_vat_exempt = vat_exclusives['Amount'].sum()
vat_exempt = vat_of_vat_exempt / 0.12
vatable = gross - vat_exempt
vat = vatable * 0.12
trans_start = valid['ExternalReference'].min()
trans_end = valid['ExternalReference'].max()

date_format = '%m/%d/%Y %I:%M %p'
report_date_from = valid['CreateDate'].min()
report_date_to = valid['CreateDate'].max()

with open(f"./output/{store_name}-z-report.txt", 'w', encoding='utf-8') as file:
    file.write(f"{'Consolidated Z Report':^42}\n")
    for col in app_vars.header.split('#'):
        file.write(f"{col:^42}\n")

    file.write("\n")
    file.write(f"{'Print Date:':<21}{datetime.today().strftime(date_format):>21}\n")
    file.write(f"{'Report Date From:':<21}{report_date_from.strftime(date_format):>21}\n")
    file.write(f"{'Report Date To:':<21}{report_date_to.strftime(date_format):>21}\n")
    file.write("=" * 42)
    file.write("\n")
    file.write(f"{'Trans Count:':<21}{count:>21}\n")
    file.write(f"{'Void Count:':<21}{cancelCount:>21}\n")
    file.write(f"{'Void Amount:':<21}{cancelGross:>21,.2f}\n")
    file.write(f"{'Change Fund:':<21}{10000:>21,.2f}\n")
    file.write(f"{'Gross Sales:':<21}{gross:>21,.2f}\n")
    file.write(f"{'Priv. Discount:':<21}{priv_disc:>21,.2f}\n")
    file.write(f"{'Other Discount:':<21}{open_disc:>21,.2f}\n")
    file.write(f"{'VAT Amount:':<21}{vat:>21,.2f}\n")
    file.write(f"{'VATABLE:':<21}{vatable:>21,.2f}\n")
    file.write(f"{'VAT Exempt:':<21}{vat_exempt:>21,.2f}\n")
    file.write(f"{'Zero-Rated:':<21}{0.00:>21,.2f}\n")
    file.write(f"{'Trans Start:':<21}{trans_start:>21}\n")
    file.write(f"{'Trans End:':<21}{trans_end:>21}\n")
    file.write(f"{'Reset Counter:':<21}{0:>21}\n")
    file.write(f"{'Checked By:':<21}{'mark':>21}\n")

with open(f'./output/{store_name}-z-report.txt', 'r', encoding='utf-8') as f:
    print(f.read())

          Consolidated Z Report           
                YANZI OPC                 
             Gil Fernando Ave             
                San Roque                 
             City of Marikina             
             NCR 2nd district             
      VAT REG TIN:604-619-459-00001       
          POS S/N: G6FN3050032B           
          Min:23072715021599188           
              TEL:8888-5436               

Print Date:            06/17/2025 12:57 AM
Report Date From:      08/06/2023 09:26 AM
Report Date To:        12/31/2023 08:51 PM
Trans Count:                          6395
Void Count:                             41
Void Amount:                     61,313.00
Change Fund:                     10,000.00
Gross Sales:                 10,536,284.00
Priv. Discount:                 447,189.51
Other Discount:                       0.00
VAT Amount:                     996,040.38
VATABLE:                      8,300,336.47
VAT Exempt:                   2,235,947.53
Zero-Rated